## Clasificación de dataset con problemas de corazón usando algoritmos Transformer

Este dataset está sacado de kaggle, en este enlace lo puedes encontrar --> <a href="https://www.kaggle.com/datasets/johnsmith88/heart-disease-dataset">Heart Disease Dataset</a>

In [6]:
# Imports
import pandas as pd
import numpy as np
#
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#
from  tabpfn import TabPFNClassifier


In [7]:
dataset = pd.read_csv('heart_statlog_cleveland_hungary_final.csv')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  1190 non-null   int64  
 1   sex                  1190 non-null   int64  
 2   chest pain type      1190 non-null   int64  
 3   resting bp s         1190 non-null   int64  
 4   cholesterol          1190 non-null   int64  
 5   fasting blood sugar  1190 non-null   int64  
 6   resting ecg          1190 non-null   int64  
 7   max heart rate       1190 non-null   int64  
 8   exercise angina      1190 non-null   int64  
 9   oldpeak              1190 non-null   float64
 10  ST slope             1190 non-null   int64  
 11  target               1190 non-null   int64  
dtypes: float64(1), int64(11)
memory usage: 111.7 KB


In [8]:
dataset.head()

,age,sex,chest pain type,resting bp s,cholesterol,fasting blood sugar,resting ecg,max heart rate,exercise angina,oldpeak,ST slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0


Tenemos en el dataset casi 1200 entradas de pacientes con o sin enfermedades cardiacas dependiento de una serie de características que podemos observar leyendo las columnas. Al final del dataset encontramos la columna target con valores 0 y 1, esto nos indica que es un problema de clasificación binaria, o están malos o no.

Observando el dataset vamos a crear un clasificador usando el algoritmo TabPFN. Es mejor que una Baseline usando un NN y mucho más rápido al estar preentrenado. Además cumplimos todas las condiciones para poder usarlo.

Un punto bueno de este clasificador es que únicamente tenemos que hacer un split en datos de entrenamiento y testing y el propio TabPFN se encarga de procesar las columnas así como de hacer los encoders que sean necesarios.

In [11]:
# Split the data into X and Y 
X = dataset.drop('target', axis=1)
Y = dataset['target']
X.shape, Y.shape

((1190, 11), (1190,))

In [13]:
# Split the data into training and testing
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((952, 11), (238, 11), (952,), (238,))

Es **MUY IMPORTANTE** que cumplamos que tenemos menos de 10 clases a diferentes, menos de 100 características y menos de 1000 puntos de entrenamiento.

In [16]:
model = TabPFNClassifier(device='cpu') # If we don't want to use GPU acceleration

model.fit(X_train, Y_train)

Y_pred, W_prob = model.predict(X_test, return_winning_probability=True)

accu_score = accuracy_score(Y_pred, Y_test)

print(f"The algorithm has an accuracy: {accu_score}")

/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-venv/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
/Users/alejandro/Documents/Repositorios/alejandromadrid/.python-venv/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentran

The algorithm has an accuracy: 0.9327731092436975


Como podemos observar, en menos de 2 segundos tenemos un clasificador con más de un 93% de accuracy para este dataset. No lo usaría como modelo definitivo pero si lo podemos marcar como una baseline o para comparación con otros modelos.